In [0]:
df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/*.csv", header=True,inferSchema=True)

In [0]:
df.write \
  .format("delta") \
  .mode("append") \
  .saveAsTable("ecommerce.bronze.bronze_events")

In [0]:
from pyspark.sql.functions import col

silver_df = spark.table("ecommerce.bronze.bronze_events") \
    .filter(col("user_session").isNotNull()) \
    .dropDuplicates()

In [0]:
silver_df.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("ecommerce.silver.silver_events")

In [0]:
gold_df = spark.table("ecommerce.silver.silver_events") \
    .filter(col("event_type") == "purchase")

In [0]:
gold_df.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("ecommerce.gold.gold_events")

In [0]:
%sql
REVOKE ALL PRIVILEGES
ON TABLE ecommerce.bronze.bronze_events
FROM `data_engineer`;

REVOKE ALL PRIVILEGES
ON TABLE ecommerce.bronze.bronze_events
FROM `data_analyst`;

In [0]:
%sql
REVOKE ALL PRIVILEGES
ON TABLE ecommerce.silver.silver_events
FROM `data_engineer`;

REVOKE ALL PRIVILEGES
ON TABLE ecommerce.silver.silver_events
FROM `data_analyst`;

In [0]:
%sql
REVOKE ALL PRIVILEGES
ON TABLE ecommerce.gold.gold_events
FROM `data_engineer`;

REVOKE ALL PRIVILEGES
ON TABLE ecommerce.gold.gold_events
FROM `data_analyst`;

In [0]:
%sql
REVOKE ALL PRIVILEGES
ON SCHEMA ecommerce.bronze
FROM `data_engineer`;

REVOKE ALL PRIVILEGES
ON SCHEMA ecommerce.bronze
FROM `data_analyst`;

REVOKE ALL PRIVILEGES
ON SCHEMA ecommerce.silver
FROM `data_engineer`;

REVOKE ALL PRIVILEGES
ON SCHEMA ecommerce.silver
FROM `data_analyst`;

REVOKE ALL PRIVILEGES
ON SCHEMA ecommerce.gold
FROM `data_engineer`;

REVOKE ALL PRIVILEGES
ON SCHEMA ecommerce.gold
FROM `data_analyst`;

In [0]:
%sql
REVOKE ALL PRIVILEGES
ON CATALOG ecommerce
FROM `data_engineer`;

REVOKE ALL PRIVILEGES
ON CATALOG ecommerce
FROM `data_analyst`;

## **Reassingning all the access again**

In [0]:
%sql
GRANT USE CATALOG
ON CATALOG ecommerce
TO `data_engineer`;

GRANT USE CATALOG
ON CATALOG ecommerce
TO `data_analyst`;

In [0]:
%sql
-- bronze schema access to only data engineer
GRANT USE SCHEMA
ON SCHEMA ecommerce.bronze
TO `data_engineer`;

-- Silver schema access to data analyst & data engineer
GRANT USE SCHEMA
ON SCHEMA ecommerce.silver
TO `data_engineer`;

GRANT USE SCHEMA
ON SCHEMA ecommerce.silver
TO `data_analyst`;

-- Gold Schema access to data analyst & data engineer
GRANT USE SCHEMA
ON SCHEMA ecommerce.gold
TO `data_engineer`;

GRANT USE SCHEMA
ON SCHEMA ecommerce.gold
TO `data_analyst`;

In [0]:
%sql
-- bronze table access to only data engineer
GRANT ALL PRIVILEGES
ON TABLE ecommerce.bronze.bronze_events
TO `data_engineer`;

-- silver table access to data analyst & data engineer
GRANT ALL PRIVILEGES
ON TABLE ecommerce.silver.silver_events
TO `data_engineer`;

GRANT SELECT
ON TABLE ecommerce.silver.silver_events
TO `data_analyst`;

-- gold table access to data analyst & data engineer
GRANT ALL PRIVILEGES
ON TABLE ecommerce.gold.gold_events
TO `data_engineer`;

GRANT SELECT
ON TABLE ecommerce.gold.gold_events
TO `data_analyst`;

In [0]:
%sql
SHOW GRANTS ON TABLE ecommerce.gold.gold_events;



Principal,ActionType,ObjectType,ObjectKey
data_analyst,SELECT,TABLE,ecommerce.gold.gold_events
data_engineer,ALL PRIVILEGES,TABLE,ecommerce.gold.gold_events


In [0]:
%sql
SELECT * FROM ecommerce.gold.gold_events LIMIT 10;

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-11-17T08:44:36.000Z,purchase,28100847,2053013564918072245,null,moldabela,79.28,519048643,281c9bca-8371-448d-a914-c3b20145c72c
2019-11-17T08:47:08.000Z,purchase,4000197,2053013566142809077,construction.tools.generator,polaris,69.47,547716044,b581d207-bd50-4626-90ff-687a8d365f8f
2019-11-17T08:49:29.000Z,purchase,28722404,2053013565782098913,apparel.shoes,respect,71.82,529450554,3180c8a6-ac8d-421b-9044-9738b4f99f0c
2019-11-17T08:49:33.000Z,purchase,1005164,2053013555631882655,electronics.smartphone,xiaomi,278.77,541896426,58206d37-485f-463c-9152-2f203ca78b2c
2019-11-17T08:56:04.000Z,purchase,17200570,2053013559792632471,furniture.living_room.sofa,null,463.07,517518779,b0152fd8-3e5b-4364-90d3-44c86f16428b
2019-11-17T09:06:44.000Z,purchase,1002544,2053013555631882655,electronics.smartphone,apple,493.96,526826491,575d47e6-a38d-41ed-a885-b8d5ba646239
2019-11-17T09:11:53.000Z,purchase,5100816,2053013553375346967,null,xiaomi,32.28,544451298,e2cd7659-f030-4c86-ad35-d070fbdf1470
2019-11-17T09:16:17.000Z,purchase,1004856,2053013555631882655,electronics.smartphone,samsung,128.42,525262884,b3ebf2ae-e21c-4ad6-aaa8-4944859812f3
2019-11-17T09:17:55.000Z,purchase,1005109,2053013555631882655,electronics.smartphone,apple,977.81,567048808,c0365277-fdc2-40ed-a98e-89a5263e89bc
2019-11-17T09:19:13.000Z,purchase,2900958,2053013554776244595,appliances.kitchen.microwave,arg,59.18,513577139,56befecf-2764-4bd9-a5bd-32871995334c


In [0]:
%sql
SHOW GRANTS ON TABLE ecommerce.silver.silver_events;

Principal,ActionType,ObjectType,ObjectKey
data_analyst,SELECT,TABLE,ecommerce.silver.silver_events
data_engineer,ALL PRIVILEGES,TABLE,ecommerce.silver.silver_events


In [0]:
%sql
SHOW GRANTS ON TABLE ecommerce.bronze.bronze_events;

Principal,ActionType,ObjectType,ObjectKey
data_engineer,ALL PRIVILEGES,TABLE,ecommerce.bronze.bronze_events
data_engineer,MANAGE,SCHEMA,ecommerce.bronze


In [0]:
%sql
CREATE OR REPLACE VIEW ecommerce.gold.vw_sales_summary AS
SELECT
event_type,
category_code,
brand,
SUM(price) AS total_revenue
FROM ecommerce.gold.gold_events
GROUP BY event_type,cate


In [0]:
%sql
GRANT SELECT
ON VIEW ecommerce.gold.vw_sales_summary
TO `data_analyst`;


In [0]:
%sql
SELECT * FROM ecommerce.gold.vw_sales_summary;

event_type,category_code,brand,total_revenue
purchase,appliances.kitchen.grill,maxwell,17208.61
purchase,accessories.bag,defender,148.03
purchase,null,can-am,3944.81
purchase,kids.toys,doubleeagle,64.33
purchase,null,electronicarts,4213.76
purchase,null,beko,61259.68000000001
purchase,null,atoll,5544.34
purchase,null,seagate,7816.63
purchase,null,mirage,9594.490000000002
purchase,null,usams,30.84
